# 0. Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import os
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import pandas as pd
import re

# 1. Web Scrapping 

The main goal of web scrapping in this website is to get the 5 top viral recipes worldwide in TikTok and their ingredients. Therefore, it would be qualitative searching.

We are going to scrapp the most popular website in Spain about recipes: : www.directoalpaladar.com

In [2]:
# URL from the website
url = 'https://www.directoalpaladar.com/actualidad-1/estas-han-sido-diez-recetas-virales-tiktok-2022'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
# We want to find the labels <h2>, as they are the ones with ores valuated information"ingredients" 
dict_recipes = {}
for i in soup.find_all('h2'):
    key = i.get_text().strip()
    value = ""
    next_node = i.next_sibling
    while next_node and next_node.name != 'h2':
        if next_node.name is not None:
            value += next_node.get_text().strip() + " "
        next_node = next_node.next_sibling
    dict_recipes[key] = value.strip()


1.1 Web scrapping to get the ingredients

In [4]:
#With the following loop we iterate 
new_dict = {}
for key, value in dict_recipes.items(): 
    # Iterate where the word "Ingredientes" appears:
    ingredients_index = value.find("Ingredientes")
    # if "Ingredientes" it's in the string, we proceed as follows:
    if ingredients_index != -1:
        # Find the index where "Elaboración" appears
        elaboration_index = value.find("Elaboración")
        # If "Elaboración" it's present, we delete what is comming after (as we are not interested in get the recipe elaboration)
        if elaboration_index != -1:
            value = value[ingredients_index:elaboration_index]
        # If not, we delete everything is after Ingredients.S
        else:
            value = value[ingredients_index:]
    # If Ingredients it's not in the text, we eliminate everything
    else:
        value = dict_recipes[key]
    new_dict[key] = value


More cleaning....

In [5]:
for key, value in dict_recipes.items():
    ingredients_index = value.find("Ingredientes")
    if ingredients_index != -1:
        dict_recipes[key] = value[ingredients_index:]
    else:
        dict_recipes[key] = ""
    elaboracion_index = dict_recipes[key].find("Elaboración")
    if elaboracion_index != -1:
        dict_recipes[key] = dict_recipes[key][:elaboracion_index]
print(dict_recipes)

{'1. Pan nube': 'Ingredientes para unas 9 unidades. 3 huevos L, 100 g de queso crema, 1/4 cucharadita de bicarbonato sódico.\n', '2. Avena horneada': 'Ingredientes.  100 g de copos de avena gruesos, 1 plátano grande o 2 medianos, 250 ml de leche o bebida vegetal, 1 cucharadita de canela, 1 cucharadita opcional de vainilla, 10 g de semillas de chía, una pizca de sal, 15 g de nueces picadas, 25 g de arándanos, ralladura de naranja opcional.\n', '3. Tabla de charcutería': '', '4. Pasta chips': 'Ingredientes para 4 personas. 500 g de pasta corta seca, queso parmesano o grana padano rallado, sal, pimienta negra, ajo granulado, chile en copos o pimentón picante, orégano y aceite de oliva virgen extra.\n', '5. Mug cake': 'Ingredientes para 1-2 unidades. 70 g de chocolate negro, 30 g de mantequilla, 1 huevo M, 10-20 g de azúcar, 20 g de harina de repostería.\n', '6. Birria tacos': '', '7. Salsa rosa': '', '8. Rollos de canela': 'Ingredientes. 325 g de harina de fuerza, 2 g de sal, 125 ml de le

Cleaning in order to have only the top 5. Therefore, we .pop:

In [6]:
remove_recipes=['6. Birria tacos','7. Salsa rosa','8. Rollos de canela','9. Tablas de nachos','10. Tablas de mantequilla']
[dict_recipes.pop(x,None) for x in remove_recipes]
dict_recipes

{'1. Pan nube': 'Ingredientes para unas 9 unidades. 3 huevos L, 100 g de queso crema, 1/4 cucharadita de bicarbonato sódico.\n',
 '2. Avena horneada': 'Ingredientes.  100 g de copos de avena gruesos, 1 plátano grande o 2 medianos, 250 ml de leche o bebida vegetal, 1 cucharadita de canela, 1 cucharadita opcional de vainilla, 10 g de semillas de chía, una pizca de sal, 15 g de nueces picadas, 25 g de arándanos, ralladura de naranja opcional.\n',
 '3. Tabla de charcutería': '',
 '4. Pasta chips': 'Ingredientes para 4 personas. 500 g de pasta corta seca, queso parmesano o grana padano rallado, sal, pimienta negra, ajo granulado, chile en copos o pimentón picante, orégano y aceite de oliva virgen extra.\n',
 '5. Mug cake': 'Ingredientes para 1-2 unidades. 70 g de chocolate negro, 30 g de mantequilla, 1 huevo M, 10-20 g de azúcar, 20 g de harina de repostería.\n'}

We decided to cleaning, assigning a list of desired words:

In [7]:
desired_words = ['huevos', 'queso', 'bicarbonato', 'avena', 'canela', 'plátano', 'chía', 'sal', 'nueces', 'arándanos', 'naranja', 'pasta', 'queso parmesano', 'pimienta', 'ajo', 'chile', 'orégano', 'aceite', 'chocolate', 'mantequilla', 'azúcar', 'harina', 'levadura', 'canela', 'agua', 'vainilla']

new_dict = {}
for key, value in dict_recipes.items():
    words = []
    for word in value.split():
        word = word.strip().strip(',.')
        if word.lower() in desired_words:
            words.append(word)
    new_dict[key] = ' '.join(words)
new_dict

{'1. Pan nube': 'huevos queso bicarbonato',
 '2. Avena horneada': 'avena plátano canela vainilla chía sal nueces arándanos naranja',
 '3. Tabla de charcutería': '',
 '4. Pasta chips': 'pasta queso sal pimienta ajo chile orégano aceite',
 '5. Mug cake': 'chocolate mantequilla azúcar harina'}

Let's separate with commas and convert into UPPER to homogenize (because the data set on the consume is in Uppers)

In [8]:
for key, value in new_dict.items():
    words = value.upper().split()
    words = ", ".join([word.upper() for word in words])
    new_dict[key] = words
print(new_dict)

{'1. Pan nube': 'HUEVOS, QUESO, BICARBONATO', '2. Avena horneada': 'AVENA, PLÁTANO, CANELA, VAINILLA, CHÍA, SAL, NUECES, ARÁNDANOS, NARANJA', '3. Tabla de charcutería': '', '4. Pasta chips': 'PASTA, QUESO, SAL, PIMIENTA, AJO, CHILE, ORÉGANO, ACEITE', '5. Mug cake': 'CHOCOLATE, MANTEQUILLA, AZÚCAR, HARINA'}


There are some missing values in the ingredients in our website, so we will try to find this data in other websites:

In [9]:
for key, value in new_dict.items():
    if value == '':
        print(f'{key}')

3. Tabla de charcutería


#### Recipe top3 - Charcuterie Board


Web scrapping, cleaning and transformation from other website: 'https://www.spendwithpennies.com/how-to-make-a-charcuterie-board/

In [10]:
url = 'https://www.spendwithpennies.com/how-to-make-a-charcuterie-board/'
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')


Find the ingredients throught the class.

In [11]:
import re
ingredient_charquterias = soup2.find_all("div", attrs = {"class": "is-layout-flow wp-block-group"})
list_ingredients_char=[]
for div in ingredient_charquterias:
    for p in div.find_all('p'):
        ingredient=re.search('<strong><em>(.*?)<\/em><\/strong>', str(p))
        if ingredient:
            list_ingredients_char.append(ingredient.group(1))     
print(list_ingredients_char)

['Meats', 'Cheese ', 'Bread &amp; Crackers', 'Fruit &amp; Nuts', 'Sweet', 'Pickles, Olives, &amp; Dips']


Cleaning weird characters (&amp):

In [12]:
cleaned_ingredients = [i.replace('&amp;', '&') for i in list_ingredients_char]
print(cleaned_ingredients)

['Meats', 'Cheese ', 'Bread & Crackers', 'Fruit & Nuts', 'Sweet', 'Pickles, Olives, & Dips']


Homogenize: Translate word into spanish, as all the data is in Spanish. We will use a library googletrans

In [13]:
from googletrans import Translator
translator = Translator()
cleaned_ingredients=['Meats', 'Cheese ', 'Bread & Crackers', 'Fruit & Nuts', 'Sweet', 'Pickles, Olives, & Dips']
# Iterate each element of cleaned_ingredients
translated_list = []
for ingredient in cleaned_ingredients:
    translated_text = translator.translate(ingredient, dest='es').text
    translated_list.append(translated_text)
print(translated_list)

['Carnes', 'Queso', 'Pan y galletas', 'Frutas y nueces', 'Dulce', 'Encurtidos, aceitunas y caídas']


Replace "y":

In [14]:
new_list = [item.replace(' y ', ', ') for item in translated_list]


Convert into singular string each word and uppercase:

In [15]:
separated_list = []
for item in new_list:
    words = item.split(",")
    for word in words:
        separated_list.append(word.strip().upper())
print(separated_list)

['CARNES', 'QUESO', 'PAN', 'GALLETAS', 'FRUTAS', 'NUECES', 'DULCE', 'ENCURTIDOS', 'ACEITUNAS', 'CAÍDAS']


In [16]:
type(separated_list)

list

Now, once we have the ingredient from the recipe 3 " Charquterie board", we need to include them in our main dictionary with the rest of recipes and ingredients.

In [17]:
new_dict['3. Tabla de charcutería']='CARNES', 'QUESO', 'PAN', 'GALLETAS', 'FRUTAS', 'NUECES', 'DULCE', 'ENCURTIDOS', 'ACEITUNAS'


In [18]:
for k, v in new_dict.items():
    if isinstance(v, tuple):
        new_dict[k] = ", ".join(list(v))

print(new_dict)


{'1. Pan nube': 'HUEVOS, QUESO, BICARBONATO', '2. Avena horneada': 'AVENA, PLÁTANO, CANELA, VAINILLA, CHÍA, SAL, NUECES, ARÁNDANOS, NARANJA', '3. Tabla de charcutería': 'CARNES, QUESO, PAN, GALLETAS, FRUTAS, NUECES, DULCE, ENCURTIDOS, ACEITUNAS', '4. Pasta chips': 'PASTA, QUESO, SAL, PIMIENTA, AJO, CHILE, ORÉGANO, ACEITE', '5. Mug cake': 'CHOCOLATE, MANTEQUILLA, AZÚCAR, HARINA'}


Create a final DF for the Recipes and their ingredients

In [19]:
import pandas as pd
df_recipes = pd.DataFrame.from_dict(new_dict, orient='index')
df_recipes = df_recipes.rename(columns={0: 'Ingredient'})
df_recipes = df_recipes.reset_index().rename(columns={'index': 'Recipe'})
df_recipes

,Recipe,Ingredient
0,1. Pan nube,"HUEVOS, QUESO, BICARBONATO"
1,2. Avena horneada,"AVENA, PLÁTANO, CANELA, VAINILLA, CHÍA, SAL, N..."
2,3. Tabla de charcutería,"CARNES, QUESO, PAN, GALLETAS, FRUTAS, NUECES, ..."
3,4. Pasta chips,"PASTA, QUESO, SAL, PIMIENTA, AJO, CHILE, ORÉGA..."
4,5. Mug cake,"CHOCOLATE, MANTEQUILLA, AZÚCAR, HARINA"


Split each ingredient of each recipe in the column Ingredients

In [20]:
df_ingredientes = df_recipes['Ingredient'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('Ingredients')
df_recipes = df_recipes.drop('Ingredient', axis=1).join(df_ingredientes).reset_index(drop=True)
df_recipes

,Recipe,Ingredients
0,1. Pan nube,HUEVOS
1,1. Pan nube,QUESO
2,1. Pan nube,BICARBONATO
3,2. Avena horneada,AVENA
4,2. Avena horneada,PLÁTANO
5,2. Avena horneada,CANELA
6,2. Avena horneada,VAINILLA
7,2. Avena horneada,CHÍA
8,2. Avena horneada,SAL
9,2. Avena horneada,NUECES


# 2. Data set - Cleaning & transformation 

In [29]:
import warnings
warnings.filterwarnings("ignore")

We take a official dataset from the Spainish Goverment with the consumtions, volumns, penetration , etc of the different aliment groups.

In [30]:
df=pd.read_csv('data/TOTALS1.csv')

In [31]:
df.columns

Index(['CONSUMO EN HOGARES', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9',
       ...
       'Unnamed: 100', 'Unnamed: 101', 'Unnamed: 102', 'Unnamed: 103',
       'Unnamed: 104', 'Unnamed: 105', 'Unnamed: 106', 'Unnamed: 107',
       'Unnamed: 108', 'Unnamed: 109'],
      dtype='object', length=110)

Transformation of headers

In [32]:
df= df.rename(columns={
                        'CONSUMO EN HOGARES': 'ALIMENT_GROUP',
                        'Unnamed: 1': 'MONTH',
                        'Unnamed: 2': 'PER_CAPITA_CONSUMTION',
                        'Unnamed: 3': 'EXPENDITURE_PER_CAPITA',
                        'Unnamed: 4': 'PENETRATION(%)',
                        'Unnamed: 5': 'AVG VOLUMN(Kg. or l.)',
                        'Unnamed: 6': 'VALUE',
                        'Unnamed: 7': 'VOLUME'})
df

,ALIMENT_GROUP,MONTH,PER_CAPITA_CONSUMTION,EXPENDITURE_PER_CAPITA,PENETRATION(%),AVG VOLUMN(Kg. or l.),VALUE,VOLUME,Unnamed: 8,Unnamed: 9,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
0,ENERO 22,NaN,T.ESPAÑA,NaN,NaN,NaN,NaN,NaN,CATALUÑA,NaN,...,NaN,NaN,NaN,NaN,CANARIAS,NaN,NaN,NaN,NaN,NaN
1,NaN,MONTH,CONSUMO X CAPITA,GASTO X CAPITA,PENETRACION (%),PRECIO MEDIO kg ó litros,VALOR (Miles Euros),VOLUMEN (Miles kg ó litros),CONSUMO X CAPITA,GASTO X CAPITA,...,PENETRACION (%),PRECIO MEDIO kg ó litros,VALOR (Miles Euros),VOLUMEN (Miles kg ó litros),CONSUMO X CAPITA,GASTO X CAPITA,PENETRACION (%),PRECIO MEDIO kg ó litros,VALOR (Miles Euros),VOLUMEN (Miles kg ó litros)
2,.TOTAL ALIMENTACION,JANUARY,48.70,126.97,100.00,2.61,"5,894,707.73","2,260,917.72",53.22,147.49,...,100.00,2.82,"83,852.55","29,691.12",51.53,111.74,100.00,2.17,"250,363.94","115,458.47"
3,T.HUEVOS KGS,JANUARY,0.67,1.71,78.13,2.55,"79,487.84","31,212.11",0.67,1.85,...,77.64,2.49,"1,298.70",520.96,0.65,1.84,71.82,2.85,"4,126.16","1,449.23"
4,T.HUEVOS UNDS.,JANUARY,10.79,1.71,78.13,0.16,"79,487.84","500,797.09",10.91,1.85,...,77.64,0.16,"1,298.70","8,257.16",10.13,1.84,71.82,0.18,"4,126.16","22,705.62"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7466,SALSAS SIN GLUTEN,NOVEMBER,0.05,0.18,18.07,3.84,"8,238.62","2,148.16",0.03,0.12,...,12.50,3.88,69.33,17.85,0.05,0.19,18.26,3.90,424.88,108.81
7467,GASEOSAS&BEBID.REF.SG,NOVEMBER,0.17,0.19,9.71,1.11,"8,776.97","7,888.54",0.10,0.12,...,6.12,3.23,64.78,20.07,0.20,0.19,11.37,0.95,421.05,441.59
7468,BEBIDAS REFRESCAN.SG,NOVEMBER,0.17,0.19,9.71,1.11,"8,776.97","7,888.54",0.10,0.12,...,6.12,3.23,64.78,20.07,0.20,0.19,11.37,0.95,421.05,441.59
7469,HARINAS Y SEMOLAS SG,NOVEMBER,0.02,0.05,4.78,3.30,"2,496.49",755.94,0.02,0.07,...,3.54,3.48,23.83,6.85,0.02,0.06,6.30,2.47,124.30,50.31


In [33]:
df= df.drop([0,1])
df


,ALIMENT_GROUP,MONTH,PER_CAPITA_CONSUMTION,EXPENDITURE_PER_CAPITA,PENETRATION(%),AVG VOLUMN(Kg. or l.),VALUE,VOLUME,Unnamed: 8,Unnamed: 9,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
2,.TOTAL ALIMENTACION,JANUARY,48.70,126.97,100.00,2.61,"5,894,707.73","2,260,917.72",53.22,147.49,...,100.00,2.82,"83,852.55","29,691.12",51.53,111.74,100.00,2.17,"250,363.94","115,458.47"
3,T.HUEVOS KGS,JANUARY,0.67,1.71,78.13,2.55,"79,487.84","31,212.11",0.67,1.85,...,77.64,2.49,"1,298.70",520.96,0.65,1.84,71.82,2.85,"4,126.16","1,449.23"
4,T.HUEVOS UNDS.,JANUARY,10.79,1.71,78.13,0.16,"79,487.84","500,797.09",10.91,1.85,...,77.64,0.16,"1,298.70","8,257.16",10.13,1.84,71.82,0.18,"4,126.16","22,705.62"
5,TOTAL HUEVOS GALLINA,JANUARY,10.45,1.68,77.84,0.16,"78,214.53","485,261.80",10.45,1.82,...,77.66,0.16,"1,286.42","8,118.24",10.10,1.84,71.57,0.18,"4,120.13","22,632.92"
6,OTRAS AVES,JANUARY,0.33,0.03,3.64,0.08,"1,273.31","15,535.29",0.45,0.04,...,3.17,0.09,12.28,138.93,0.03,0.00,0.59,0.08,6.03,72.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7466,SALSAS SIN GLUTEN,NOVEMBER,0.05,0.18,18.07,3.84,"8,238.62","2,148.16",0.03,0.12,...,12.50,3.88,69.33,17.85,0.05,0.19,18.26,3.90,424.88,108.81
7467,GASEOSAS&BEBID.REF.SG,NOVEMBER,0.17,0.19,9.71,1.11,"8,776.97","7,888.54",0.10,0.12,...,6.12,3.23,64.78,20.07,0.20,0.19,11.37,0.95,421.05,441.59
7468,BEBIDAS REFRESCAN.SG,NOVEMBER,0.17,0.19,9.71,1.11,"8,776.97","7,888.54",0.10,0.12,...,6.12,3.23,64.78,20.07,0.20,0.19,11.37,0.95,421.05,441.59
7469,HARINAS Y SEMOLAS SG,NOVEMBER,0.02,0.05,4.78,3.30,"2,496.49",755.94,0.02,0.07,...,3.54,3.48,23.83,6.85,0.02,0.06,6.30,2.47,124.30,50.31


Concatenate the values from the web scrapping to the dataset in order to verify the aliment grous affected by the viral recipes

In [34]:
vol_ES_df = df.iloc[:, 0:8]
df = pd.concat([vol_ES_df, df_recipes], ignore_index=True)
df

,ALIMENT_GROUP,MONTH,PER_CAPITA_CONSUMTION,EXPENDITURE_PER_CAPITA,PENETRATION(%),AVG VOLUMN(Kg. or l.),VALUE,VOLUME,Recipe,Ingredients
0,.TOTAL ALIMENTACION,JANUARY,48.70,126.97,100.00,2.61,"5,894,707.73","2,260,917.72",NaN,NaN
1,T.HUEVOS KGS,JANUARY,0.67,1.71,78.13,2.55,"79,487.84","31,212.11",NaN,NaN
2,T.HUEVOS UNDS.,JANUARY,10.79,1.71,78.13,0.16,"79,487.84","500,797.09",NaN,NaN
3,TOTAL HUEVOS GALLINA,JANUARY,10.45,1.68,77.84,0.16,"78,214.53","485,261.80",NaN,NaN
4,OTRAS AVES,JANUARY,0.33,0.03,3.64,0.08,"1,273.31","15,535.29",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4. Pasta chips,ACEITE
7498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5. Mug cake,CHOCOLATE
7499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5. Mug cake,MANTEQUILLA
7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5. Mug cake,AZÚCAR


changing data type

In [35]:
df['Ingredients'] = df['Ingredients'].astype(str)

In [36]:
df['ALIMENT_GROUP'] = df['ALIMENT_GROUP'].astype(str)

Creating a list to correlate each ingredient of the recipes with the aliment group in the data set. If the ingredint has correlation, we will add de Aliment Group in the column.

In [37]:
ingredientes_dict = {'HUEVOS': 'T.HUEVOS KGS', 'LECHE': 'T.LECHE LT', 'ACEITE':'TOTAL ACEITE','QUESO':'QUESO','BICARBONATO':'ESPECIAS Y CONDIMENTO','AVENA':'CEREALES','PLÁTANO':'T.FRUTAS FRESCAS','CHÍA': 'CEREALES','VAINILLA':'ESPECIAS Y CONDIMENTO','SAL':'ESPECIAS Y CONDIMENTO','NUECES':'FRUTOS SEC.NUECES','ENCURTIDOS':'ENCURTIDOS','ARÁNDANOS':'T.FRUTAS FRESCAS','NARANJA':'T.FRUTAS FRESCAS','AZÚCAR':'AZUCAR','CARNES':'TOTAL CARNE','PAN':'PAN','GALLETAS':'GALLETAS','FRUTAS':'T.FRUTAS FRESCAS','ACEITUNAS':'ACEITUNAS','PASTA':'PASTAS ALIMENTICIAS','PIMIENTA':'ESPECIAS Y CONDIMENTO','CANELA':'ESPECIAS Y CONDIMENTO','AJO':'T.HORTALIZAS FRESCAS','CHILE':'T.FRUTAS FRESCAS','DULCE':'T.FRUTAS FRESCAS','ORÉGANO':'ESPECIAS Y CONDIMENTO','CHOCOLATE':'CHOCOLATES','MANTEQUILLA':'MANTEQUILLA','HARINA':'HARINAS Y SEMOLAS'}

for index, row in df.iterrows():
    for ingrediente, grupo in ingredientes_dict.items():
        if ingrediente in row['Ingredients']:
            df.at[index, 'ALIMENT_GROUP'] = grupo


Replace whitespaces i some rows in order to count as Null values to better filtering after:

In [38]:
import numpy as np

df['Ingredients'].replace('', np.nan, inplace=True)
subset = df.dropna(subset=['Ingredients'])


In [39]:
import numpy as np

df['Ingredients'].replace('nan', np.nan, inplace=True)
subset = df.dropna(subset=['Ingredients'])


We select only the value sthat we want to analyse. Therefore, only the aliment groups that are viral:

In [40]:
ingredientes_dict = {
    'T.HUEVOS KGS', 'T.LECHE LT', 'TOTAL ACEITE', 'QUESO', 'ESPECIAS Y CONDIMENTO', 'CEREALES',
    'T.FRUTAS FRESCAS', 'FRUTOS SEC.NUECES', 'ENCURTIDOS', 'AZUCAR', 'TOTAL CARNE', 'PAN',
    'GALLETAS', 'ACEITUNAS', 'PASTAS ALIMENTICIAS', 'CHOCOLATES', 'MANTEQUILLA', 'HARINAS Y SEMOLAS'
}

# Crear nuevo DataFrame solo con valores permitidos
df_filtrado = df[df['ALIMENT_GROUP'].isin(ingredientes_dict)].copy()
# Eliminar filas donde MONTH sea nulo
df_filtrado.dropna(subset=['MONTH'], inplace=True)
# Imprimir el nuevo DataFrame
df_filtrado

,ALIMENT_GROUP,MONTH,PER_CAPITA_CONSUMTION,EXPENDITURE_PER_CAPITA,PENETRATION(%),AVG VOLUMN(Kg. or l.),VALUE,VOLUME,Recipe,Ingredients
1,T.HUEVOS KGS,JANUARY,0.67,1.71,78.13,2.55,"79,487.84","31,212.11",NaN,NaN
8,TOTAL CARNE,JANUARY,3.55,25.54,97.38,7.19,"1,185,557.68","164,861.56",NaN,NaN
222,MANTEQUILLA,JANUARY,0.03,0.26,20.36,8.14,"12,165.60","1,494.92",NaN,NaN
225,QUESO,JANUARY,0.62,4.83,87.09,7.81,"224,340.14","28,724.90",NaN,NaN
254,PAN,JANUARY,2.47,5.86,97.07,2.37,"272,006.47","114,808.39",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7243,T.FRUTAS FRESCAS,NOVEMBER,5.92,11.96,93.81,2.02,"555,291.64","274,721.68",NaN,NaN
7291,FRUTOS SEC.NUECES,NOVEMBER,0.06,0.54,20.88,8.39,"25,272.07","3,011.28",NaN,NaN
7399,HARINAS Y SEMOLAS,NOVEMBER,0.18,0.23,27.25,1.32,"10,873.00","8,211.95",NaN,NaN
7400,ENCURTIDOS,NOVEMBER,0.04,0.16,14.62,3.70,"7,610.45","2,058.40",NaN,NaN


# 3. VISUALIZATION

Create a subset to visualizate the most viral group aliments

In [41]:
subset1 = df[df['Ingredients'].notnull()]


In [42]:
df_filtrado['VOLUME'] = df_filtrado['VOLUME'].astype(str).str.split('.', expand=True)[0]
df_filtrado["VOLUME"] = df_filtrado["VOLUME"].str.replace(",", "").astype(int)
df_filtrado["VOLUME"] = df_filtrado["VOLUME"].astype(int)


In [43]:
df_filtrado['VALUE'] = df_filtrado['VALUE'].astype(str).str.split('.', expand=True)[0]
df_filtrado["VALUE"] = df_filtrado["VALUE"].str.replace(",", "").astype(int)
df_filtrado["VALUE"] = df_filtrado["VALUE"].astype(int)
df_filtrado

,ALIMENT_GROUP,MONTH,PER_CAPITA_CONSUMTION,EXPENDITURE_PER_CAPITA,PENETRATION(%),AVG VOLUMN(Kg. or l.),VALUE,VOLUME,Recipe,Ingredients
1,T.HUEVOS KGS,JANUARY,0.67,1.71,78.13,2.55,79487,31212,NaN,NaN
8,TOTAL CARNE,JANUARY,3.55,25.54,97.38,7.19,1185557,164861,NaN,NaN
222,MANTEQUILLA,JANUARY,0.03,0.26,20.36,8.14,12165,1494,NaN,NaN
225,QUESO,JANUARY,0.62,4.83,87.09,7.81,224340,28724,NaN,NaN
254,PAN,JANUARY,2.47,5.86,97.07,2.37,272006,114808,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7243,T.FRUTAS FRESCAS,NOVEMBER,5.92,11.96,93.81,2.02,555291,274721,NaN,NaN
7291,FRUTOS SEC.NUECES,NOVEMBER,0.06,0.54,20.88,8.39,25272,3011,NaN,NaN
7399,HARINAS Y SEMOLAS,NOVEMBER,0.18,0.23,27.25,1.32,10873,8211,NaN,NaN
7400,ENCURTIDOS,NOVEMBER,0.04,0.16,14.62,3.70,7610,2058,NaN,NaN


In [44]:
df_filtrado['PENETRATION(%)'] = df_filtrado['PENETRATION(%)'].astype(str).str.split('.', expand=True)[0]
df_filtrado["PENETRATION(%)"] = df_filtrado["PENETRATION(%)"].astype(int)
df_filtrado

,ALIMENT_GROUP,MONTH,PER_CAPITA_CONSUMTION,EXPENDITURE_PER_CAPITA,PENETRATION(%),AVG VOLUMN(Kg. or l.),VALUE,VOLUME,Recipe,Ingredients
1,T.HUEVOS KGS,JANUARY,0.67,1.71,78,2.55,79487,31212,NaN,NaN
8,TOTAL CARNE,JANUARY,3.55,25.54,97,7.19,1185557,164861,NaN,NaN
222,MANTEQUILLA,JANUARY,0.03,0.26,20,8.14,12165,1494,NaN,NaN
225,QUESO,JANUARY,0.62,4.83,87,7.81,224340,28724,NaN,NaN
254,PAN,JANUARY,2.47,5.86,97,2.37,272006,114808,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7243,T.FRUTAS FRESCAS,NOVEMBER,5.92,11.96,93,2.02,555291,274721,NaN,NaN
7291,FRUTOS SEC.NUECES,NOVEMBER,0.06,0.54,20,8.39,25272,3011,NaN,NaN
7399,HARINAS Y SEMOLAS,NOVEMBER,0.18,0.23,27,1.32,10873,8211,NaN,NaN
7400,ENCURTIDOS,NOVEMBER,0.04,0.16,14,3.70,7610,2058,NaN,NaN


In [48]:
import plotly.express as px
import plotly.io as pio
pio.write_html(fig, file='frequency.html', auto_open=True)
counts = subset1.groupby("ALIMENT_GROUP").size().reset_index(name="count")
counts = counts.sort_values('count', ascending=False)  
fig = px.bar(counts, x="count", y="ALIMENT_GROUP", orientation="h", color="ALIMENT_GROUP",
             color_discrete_sequence=px.colors.qualitative.Dark2)
fig.update_layout(
    title="Food ingredients presence",
    xaxis_title="Frequency ingredient viral recipes",
    yaxis_title="Category food groups"
)
fig.show()


In [49]:
subset_pan = df_filtrado[df_filtrado['ALIMENT_GROUP'].isin(['ESPECIAS Y CONDIMENTO', 'T.HUEVOS KGS', 'QUESO'])]


In [51]:
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.write_html(fig, file='annual consumption.html', auto_open=True)
grouped = df_filtrado.groupby(['ALIMENT_GROUP'])['VOLUME'].sum().sort_values(ascending=False)
colors = px.colors.qualitative.Plotly
fig = go.Figure([go.Bar(x=grouped.index, y=grouped.values, marker_color=colors)])
fig.update_layout(title='Annual consumption x aliment group', xaxis_title='Food category',
                  yaxis_title='Volum (Thousand of Kg. o L)', xaxis_tickangle=-90)



In [53]:
import plotly.express as px

fig = px.scatter(df_filtrado, x="VALUE", y="VOLUME", color="ALIMENT_GROUP", size="VOLUME",
                 color_discrete_sequence=px.colors.qualitative.Alphabet)


In [ ]:
import plotly.express as px
import plotly.io as pio
pio.write_html(fig, file='monthly consumption.htlm', auto_open=True)
fig = px.line(df_filtrado, x="MONTH", y="VOLUME", color="ALIMENT_GROUP")
fig.update_layout(
    title="Monthy Consumption",
    xaxis_title="Month",
    yaxis_title="Volume (Kg or L)",
    legend=dict(
        x=1.02,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="White",
        bordercolor="Grey",
        borderwidth=2
    ),
    xaxis=dict(tickangle=-90)
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.write_html(fig,file="penetration.html",auto_open=True)
df_filtrado['MONTH'] = pd.to_datetime(df_filtrado['MONTH'], format='%B') #Convert to datatime

df_filtrado = df_filtrado.sort_values('MONTH') #Order by month

# Group by "ALIMENT_GROUP" and "MONTH" calculate mean of penetration
grouped = df_filtrado.groupby(['ALIMENT_GROUP', 'MONTH'])['PENETRATION(%)'].mean().reset_index()

# Plotly
fig = px.line(grouped, x='MONTH', y='PENETRATION(%)', color='ALIMENT_GROUP', 
              title='% Monthly Market penetration vs Aliments Group')

fig.update_xaxes(
    tickangle=-45,
    tickformat='%B'
)
plt.savefig('figures/penetration.png')
os.system("start figures/penetration.png")
fig.show()


# 4. GOOGLE TRENDS

Hereby we are going to compare the google search for the top1 viral recipe. The words searched will be in engligh and spanish:
- Pan nube
- Cloud bread

In [ ]:
df_nube=pd.read_csv('data/trends_pannube.csv')

We will transfor the csv from Google trens, renaming columns and swithching weeks into months

In [ ]:
df_nube=pd.read_csv('data/pannube_cloudbread.csv')
df_nube= df_nube.rename(columns={
                        'Categoría: Todas las categorías': 'Cloud bread'})
df_nube.reset_index(inplace=True)
df_nube.rename(columns={'index': 'Weeks'}, inplace=True)
df_nube = df_nube.drop(0)
new_names = {'level_0': 'Weeks_', 'level_1': 'Search_pan_nube', 'Categoría: Todas las categorías': 'Cloud bread'}
df_nube = df_nube.rename(columns=new_names)
df_nube['Search_pan_nube'] = df_nube['Search_pan_nube'].astype(int)
print(df_nube['Search_pan_nube'].dtype)
df_nube['Cloud bread'] = df_nube['Cloud bread'].astype(int)
print(df_nube['Cloud bread'].dtype)
df_nube['Weeks_'] = pd.to_datetime(df_nube['Weeks_'])
df_nube['Month'] = df_nube['Weeks_'].dt.month_name()
df_nube_month = df_nube.groupby('Month', as_index=False)[['Search_pan_nube', 'Cloud bread']].sum()
df_nube_month

In [ ]:
# Convert the column month in an ordered categorical column 
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df_nube_month['Month'] = pd.Categorical(df_nube_month['Month'], categories=month_order, ordered=True)
# order df
df_nube_month = df_nube_month.sort_values('Month')
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_nube_month['Month'], y=df_nube_month['Search_pan_nube'], mode='lines', fill='tozeroy', name='Search_pan_nube'))
fig.add_trace(go.Scatter(x=df_nube_month['Month'], y=df_nube_month['Cloud bread'], mode='lines', fill='tozeroy', name='Cloud bread'))
fig.update_layout(title='Monthly Searches for "Pan Nube" and "Cloud Bread"',
                  xaxis_title='Month',
                  yaxis_title='Searches')
fig.show()
